# 🔤 Análisis de NLP Tradicional

Este notebook implementa las técnicas tradicionales de procesamiento de lenguaje natural.

## Objetivos:
1. **Preprocessing**: Tokenización, limpieza, normalización
2. **Análisis de frecuencias**: Palabras más comunes, n-gramas
3. **Análisis de sentimientos**: Usando VADER y TextBlob
4. **Visualizaciones**: Word clouds, gráficos de barras
5. **Modelo baseline**: Clasificador con TF-IDF

In [1]:
# Importar todas las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

# NLP libraries
import nltk
import re
from collections import Counter
from textblob import TextBlob

# Scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline

# Descargar recursos de NLTK
nltk_downloads = ['punkt', 'stopwords', 'wordnet', 'vader_lexicon', 'omw-1.4']
for item in nltk_downloads:
    try:
        nltk.download(item, quiet=True)
    except:
        pass

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

print("✅ Librerías de NLP importadas correctamente")

✅ Librerías de NLP importadas correctamente


## 1. Cargar Dataset

In [2]:
# Cargar dataset procesado
try:
    df = pd.read_csv('../data/dataset_procesado.csv')
    print(f"✅ Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
except FileNotFoundError:
    print("❌ No se encontró el dataset procesado")
    print("💡 Ejecuta primero el notebook 01_exploracion_datos.ipynb")
    
# Mostrar información básica
print("\n=== INFORMACIÓN DEL DATASET ===")
print(df.info())
print("\n=== DISTRIBUCIÓN DE SENTIMIENTOS ===")
print(df['sentiment_label'].value_counts())

✅ Dataset cargado: 1000 filas, 5 columnas

=== INFORMACIÓN DEL DATASET ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   text             1000 non-null   object
 1   sentiment        1000 non-null   int64 
 2   sentiment_label  1000 non-null   object
 3   text_length      1000 non-null   int64 
 4   word_count       1000 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 39.2+ KB
None

=== DISTRIBUCIÓN DE SENTIMIENTOS ===
sentiment_label
Negativo    400
Positivo    400
Neutral     200
Name: count, dtype: int64


## 2. Preprocessing de Texto

In [3]:
# Función de preprocessing
def preprocess_text(text):
    """
    Aplica preprocessing básico al texto:
    - Convierte a minúsculas
    - Elimina URLs, menciones, hashtags
    - Elimina caracteres especiales
    - Tokeniza y elimina stopwords
    - Aplica lemmatización
    """
    # Inicializar lemmatizer
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    # Convertir a minúsculas
    text = text.lower()
    
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Eliminar menciones y hashtags
    text = re.sub(r'@\w+|#\w+', '', text)
    
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenizar
    tokens = word_tokenize(text)
    
    # Eliminar stopwords y palabras muy cortas
    tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
    
    # Lemmatización
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

# Aplicar preprocessing
print("🔄 Aplicando preprocessing...")
df['text_clean'] = df['text'].apply(preprocess_text)

# Mostrar ejemplos
print("\n=== EJEMPLOS DE PREPROCESSING ===")
for i in range(3):
    print(f"\n{i+1}. Original: {df['text'].iloc[i][:100]}...")
    print(f"   Procesado: {df['text_clean'].iloc[i][:100]}...")

print("\n✅ Preprocessing completado")

🔄 Aplicando preprocessing...

=== EJEMPLOS DE PREPROCESSING ===

1. Original: I hate this movie, it was terrible...
   Procesado: hate movie terrible...

2. Original: Amazing film! Loved every minute of it...
   Procesado: amazing film loved every minute...

3. Original: Worst experience ever, very disappointed...
   Procesado: worst experience ever disappointed...

✅ Preprocessing completado


## 3. Análisis de Frecuencias

In [4]:
# Análisis de palabras más frecuentes
def get_top_words(texts, n=20):
    """
    Obtiene las palabras más frecuentes
    """
    all_words = ' '.join(texts).split()
    word_freq = Counter(all_words)
    return word_freq.most_common(n)

# Palabras más frecuentes por sentimiento
sentiments = ['Positivo', 'Negativo', 'Neutral']
top_words_by_sentiment = {}

for sentiment in sentiments:
    if sentiment in df['sentiment_label'].values:
        texts = df[df['sentiment_label'] == sentiment]['text_clean'].tolist()
        top_words = get_top_words(texts, 15)
        top_words_by_sentiment[sentiment] = top_words
        
        print(f"\n=== TOP 15 PALABRAS - {sentiment.upper()} ===")
        for word, count in top_words:
            print(f"{word:15} : {count:4d}")

# Palabras más frecuentes en general
print("\n=== TOP 20 PALABRAS GENERALES ===")
general_top_words = get_top_words(df['text_clean'].tolist(), 20)
for word, count in general_top_words:
    print(f"{word:15} : {count:4d}")


=== TOP 15 PALABRAS - POSITIVO ===
amazing         :  100
film            :  100
loved           :  100
every           :  100
minute          :  100
fantastic       :  100
best            :  100
movie           :  100
seen            :  100
year            :  100
excellent       :  100
quality         :  100
highly          :  100
recommend       :  100
outstanding     :  100

=== TOP 15 PALABRAS - NEGATIVO ===
hate            :  100
movie           :  100
terrible        :  100
worst           :  100
experience      :  100
ever            :  100
disappointed    :  100
bad             :  100
would           :  100
recommend       :  100
boring          :  100
predictable     :  100
plot            :  100

=== TOP 15 PALABRAS - NEUTRAL ===
okay            :  100
nothing         :  100
special         :  100
average         :  100
movie           :  100
bad             :  100
good            :  100

=== TOP 20 PALABRAS GENERALES ===
movie           :  300
bad             :  200
recomme

## 4. Modelo Baseline con TF-IDF

In [5]:
# Preparar datos para el modelo
print("🔄 Preparando datos para el modelo...")

# Usar texto limpio
X = df['text_clean']
y = df['sentiment_label']

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Datos de entrenamiento: {len(X_train)}")
print(f"Datos de prueba: {len(X_test)}")
print(f"Distribución de entrenamiento:")
print(y_train.value_counts())

🔄 Preparando datos para el modelo...
Datos de entrenamiento: 800
Datos de prueba: 200
Distribución de entrenamiento:
sentiment_label
Negativo    320
Positivo    320
Neutral     160
Name: count, dtype: int64


In [6]:
# Crear pipeline con TF-IDF + Naive Bayes
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2),
        stop_words='english'
    )),
    ('classifier', MultinomialNB())
])

# Entrenar modelo
print("🔄 Entrenando modelo Naive Bayes...")
pipeline_nb.fit(X_train, y_train)

# Hacer predicciones
y_pred_nb = pipeline_nb.predict(X_test)

# Evaluar
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"\n✅ Accuracy Naive Bayes: {accuracy_nb:.3f}")

print("\n=== REPORTE DE CLASIFICACIÓN - NAIVE BAYES ===")
print(classification_report(y_test, y_pred_nb))

🔄 Entrenando modelo Naive Bayes...

✅ Accuracy Naive Bayes: 1.000

=== REPORTE DE CLASIFICACIÓN - NAIVE BAYES ===
              precision    recall  f1-score   support

    Negativo       1.00      1.00      1.00        80
     Neutral       1.00      1.00      1.00        40
    Positivo       1.00      1.00      1.00        80

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



## 5. Guardado de Modelos

In [7]:
# Guardar modelos y resultados
import joblib
import os

# Crear directorio para modelos
os.makedirs('../data/models', exist_ok=True)

# Guardar modelo
joblib.dump(pipeline_nb, '../data/models/naive_bayes_model.pkl')

print("\n✅ Modelo guardado en '../data/models/'")
print("\n=== RESUMEN FINAL ===")
print(f"Modelo: Naive Bayes con TF-IDF")
print(f"Accuracy: {accuracy_nb:.3f}")
print(f"Dataset procesado: {len(df)} textos")
print(f"Características extraídas: TF-IDF con {pipeline_nb.named_steps['tfidf'].max_features} features")
print("\n🎉 Análisis de NLP tradicional completado!")


✅ Modelo guardado en '../data/models/'

=== RESUMEN FINAL ===
Modelo: Naive Bayes con TF-IDF
Accuracy: 1.000
Dataset procesado: 1000 textos
Características extraídas: TF-IDF con 5000 features

🎉 Análisis de NLP tradicional completado!
